In [ ]:
import numpy as np
import pandas as pd

import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'scripts'))
from edge_capacitance_inquiry import Config, main

cfg = Config(
    period_name="2001-2005",
    target_u="M1",   # change freely (M/D/R)
    target_v="R9",
    top_k_changes=0
)

impact_df = main(cfg)

impact_df.head()


,m,dt,rq,triplet,d_cap_pair_norm,d_cap_edge_strength,pair_count_base,pair_count_new,touch,touch_u,touch_v,direct_edge_added,min_dist_either,min_dist_to_M1,min_dist_to_R9,sum_min_dist
0,M4,D6,R4,M4-D6-R4,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0,inf,inf
1,M4,D6,R2,M4-D6-R2,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0,inf,inf
2,M4,D6,R1,M4-D6-R1,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0,inf,inf
3,M4,D6,R7,M4-D6-R7,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0,inf,inf
4,M4,D6,R3,M4-D6-R3,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0,inf,inf


# Plots

In [15]:
print(len(impact_df), "rows in total", f"(8*9*13) = {8*9*13})")
print(impact_df.isna().sum())

936 rows in total (8*9*13) = 936)
m                      0
dt                     0
rq                     0
triplet                0
d_cap_pair_norm        0
d_cap_edge_strength    0
pair_count_base        0
pair_count_new         0
touch                  0
touch_u                0
touch_v                0
direct_edge_added      0
min_dist_either        0
min_dist_to_M1         0
min_dist_to_R9         0
sum_min_dist           0
dtype: int64


In [16]:
import numpy as np
import pandas as pd
import plotly.express as px

# ---------------------------------------------------------------------
# Assumes you already have impact_df from the script's main(cfg)
# impact_df columns used:
#   - d_cap_pair_norm, d_cap_edge_strength
#   - touch, touch_u, touch_v, direct_edge_added
#   - min_dist_either
# ---------------------------------------------------------------------

df = impact_df.copy()

# Keep rows where both deltas exist (you can relax this if you want)
df = df[df["d_cap_pair_norm"].notna() & df["d_cap_edge_strength"].notna()].copy()

def dist_bin(x):
    if not np.isfinite(x):
        return "inf/NA"
    x = int(x)
    if x == 0:
        return "0 (contains endpoint)"
    if x == 1:
        return "1 (neighbor)"
    if x == 2:
        return "2"
    return "3+"

df["dist_bin"] = df["min_dist_either"].apply(dist_bin)

def relation_label(row):
    if row["direct_edge_added"] == 1:
        return "contains BOTH endpoints (direct)"
    if row["touch"] == 1:
        return "contains one endpoint"
    return "contains none"
df["relation"] = df.apply(relation_label, axis=1)

# Helpful hover label (if you have triplet string column already, this will use it)
if "triplet" not in df.columns:
    df["triplet"] = df.apply(lambda r: f"({r.get('m','?')},{r.get('dt','?')},{r.get('rq','?')})", axis=1)

# -------------------------
# Plot 1: Scatter distance vs Δcap_pair_norm (jittered)
# -------------------------
rng = np.random.default_rng(42)

def _jitter(s, scale):
    return s + rng.normal(0, scale, size=len(s))

# small x jitter so integer distances spread horizontally
x_jitter_scale = 0.12
# y jitter proportional to the y-range so it's visible but small
y1_jitter_scale = (df["d_cap_pair_norm"].max() - df["d_cap_pair_norm"].min()) * 0.01

df["_min_dist_j1"] = _jitter(df["min_dist_either"], x_jitter_scale)
df["_d_cap_pair_j"] = _jitter(df["d_cap_pair_norm"], y1_jitter_scale)

fig1 = px.scatter(
    df,
    x="_min_dist_j1",
    y="_d_cap_pair_j",
    color="relation",
    hover_data=["triplet", "touch", "direct_edge_added", "min_dist_either"],
    title="Distance to target edge endpoints vs Δ cap_pair_norm (jittered)",
)
fig1.update_traces(marker=dict(size=6, opacity=0.8))
fig1.add_hline(y=0)
fig1.update_layout(xaxis_title="min_dist_either (jittered for visibility)", yaxis_title="d_cap_pair_norm")
fig1.show()

# -------------------------
# Plot 2: Scatter distance vs Δcap_edge_strength (jittered)
# -------------------------
# reuse same x jitter scale; compute y jitter relative to this metric's range
y2_jitter_scale = (df["d_cap_edge_strength"].max() - df["d_cap_edge_strength"].min()) * 0.01

df["_min_dist_j2"] = _jitter(df["min_dist_either"], x_jitter_scale)
df["_d_cap_edge_j"] = _jitter(df["d_cap_edge_strength"], y2_jitter_scale)

fig2 = px.scatter(
    df,
    x="_min_dist_j2",
    y="_d_cap_edge_j",
    color="relation",
    hover_data=["triplet", "touch", "direct_edge_added", "min_dist_either"],
    title="Distance to target edge endpoints vs Δ cap_edge_strength (jittered)",
)
fig2.update_traces(marker=dict(size=6, opacity=0.8))
fig2.add_hline(y=0)
fig2.update_layout(xaxis_title="min_dist_either (jittered for visibility)", yaxis_title="d_cap_edge_strength")
fig2.show()

# -------------------------
# Plot 3: Box plot by distance bin (Δcap_pair_norm)
# -------------------------
dist_order = ["0 (contains endpoint)", "1 (neighbor)", "2", "3+", "inf/NA"]
fig3 = px.box(
    df,
    x="dist_bin",
    y="d_cap_pair_norm",
    category_orders={"dist_bin": dist_order},
    points=False,
    title="Δ cap_pair_norm by distance bin",
)
fig3.add_hline(y=0)
fig3.show()

# -------------------------
# Plot 4: Violin plot by touch (Δcap_edge_strength)
# -------------------------
touch_order = [0.0, 1.0, 2.0]
df["touch_str"] = df["touch"].map({0.0: "touch=0", 1.0: "touch=1", 2.0: "touch=2"}).fillna(df["touch"].astype(str))

fig4 = px.violin(
    df,
    x="touch_str",
    y="d_cap_edge_strength",
    box=True,
    points=False,
    category_orders={"touch_str": ["touch=0", "touch=1", "touch=2"]},
    title="Δ cap_edge_strength by touch category",
)
fig4.add_hline(y=0)
fig4.show()

# -------------------------
# Plot 5: 2D view — Δcap_pair_norm vs Δcap_edge_strength
#         (color = relation, size = |Δcap_pair_norm| + |Δcap_edge_strength|)
# -------------------------
df["impact_size"] = (df["d_cap_pair_norm"].abs() + df["d_cap_edge_strength"].abs()).astype(float)

fig5 = px.scatter(
    df,
    x="d_cap_pair_norm",
    y="d_cap_edge_strength",
    color="relation",
    size="impact_size",
    hover_data=["triplet", "touch", "direct_edge_added", "min_dist_either", "dist_bin"],
    title="Joint impact: Δ cap_pair_norm vs Δ cap_edge_strength",
)
fig5.add_vline(x=0)
fig5.add_hline(y=0)
fig5.show()


In [17]:
df[df['min_dist_either'] == 1]

,m,dt,rq,triplet,d_cap_pair_norm,d_cap_edge_strength,pair_count_base,pair_count_new,touch,touch_u,...,min_dist_to_R9,sum_min_dist,dist_bin,relation,_min_dist_j1,_d_cap_pair_j,_min_dist_j2,_d_cap_edge_j,touch_str,impact_size
0,M4,D6,R4,M4-D6-R4,0.000000,0.000000,1.0,1.0,0.0,0.0,...,1.0,2.0,1 (neighbor),contains none,1.036566,-0.031673,0.926057,0.000001,touch=0,0.000000
1,M4,D6,R2,M4-D6-R2,0.000000,0.000000,1.0,1.0,0.0,0.0,...,1.0,2.0,1 (neighbor),contains none,0.875202,-0.002418,0.856676,-0.000005,touch=0,0.000000
2,M4,D6,R1,M4-D6-R1,0.000000,0.000000,1.0,1.0,0.0,0.0,...,1.0,2.0,1 (neighbor),contains none,1.090054,0.025878,0.872263,-0.000002,touch=0,0.000000
3,M4,D6,R7,M4-D6-R7,0.000000,0.000000,1.0,1.0,0.0,0.0,...,1.0,2.0,1 (neighbor),contains none,1.112868,0.023833,0.780593,-0.000001,touch=0,0.000000
4,M4,D6,R3,M4-D6-R3,0.000000,0.000000,1.0,1.0,0.0,0.0,...,1.0,2.0,1 (neighbor),contains none,0.765876,0.005824,1.087579,-0.000002,touch=0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,M7,D9,R4,M7-D9-R4,-0.434895,-0.000058,1.0,1.0,0.0,0.0,...,2.0,3.0,1 (neighbor),contains none,1.049655,-0.406711,0.922773,-0.000059,touch=0,0.434953
928,M7,D9,R2,M7-D9-R2,-0.434895,-0.000058,1.0,1.0,0.0,0.0,...,2.0,3.0,1 (neighbor),contains none,0.972128,-0.426917,0.968846,-0.000059,touch=0,0.434953
929,M7,D9,R1,M7-D9-R1,-0.434895,-0.000058,1.0,1.0,0.0,0.0,...,2.0,3.0,1 (neighbor),contains none,1.009086,-0.430568,1.116377,-0.000052,touch=0,0.434953
930,M7,D9,R7,M7-D9-R7,-0.434895,-0.000058,1.0,1.0,0.0,0.0,...,2.0,3.0,1 (neighbor),contains none,1.000722,-0.440648,1.104913,-0.000060,touch=0,0.434953


# Impact of 1 triplet on the network

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'scripts'))

from edge_capacitance_inquiry import network_impact_of_one_triplet
import data_loads

trip = ("M3", "D13", "R5")  # example
out = network_impact_of_one_triplet(
    df_main=data_loads.main,
    period_name="2006-2010",
    triplet_key=trip,
    delta=+1.0,
    top_k=40,
    restrict_to_two_edges=False,  # set True if you *really* mean “2 edges only”
)

# out["summary"]
# out["top_changed"].head(20)      # biggest network-wide shifts
# out["delta_all"]                 # Δcap for every edge in the period graph
out["delta_touched"]             # Δcap for edges directly incremented by the triplet


,u,v,pair_count_base,pair_count_new,cap_pair_norm_base,cap_pair_norm_new,d_cap_pair_norm,edge_strength_base,edge_strength_new,cap_edge_strength_base,cap_edge_strength_new,d_cap_edge_strength,cap_unweighted_edge_base,cap_unweighted_edge_new,d_cap_unweighted_edge,edge_degree_base,edge_degree_new
14,D13,M3,1.0,2.0,0.000000,0.136364,0.136364,36.0,38.0,0.000000,0.000312,0.000312,0.000000,0.000000,0.0,15.0,15.0
21,D13,R5,1.0,2.0,1.818182,0.909091,-0.909091,24.0,26.0,0.003294,0.003040,-0.000253,0.005107,0.005107,0.0,10.0,10.0
55,M3,R5,1.0,2.0,0.181818,0.090909,-0.090909,14.0,16.0,0.000565,0.000494,-0.000071,0.005126,0.005126,0.0,7.0,7.0


In [34]:
out["summary"]

{'period': '2006-2010',
 'triplet': ('M3', 'D13', 'R5'),
 'restrict_to_two_edges': False,
 'n_edges_baseline': 64,
 'n_edges_new': 64,
 'n_edges_in_union': 64,
 'abs_sum_d_cap_pair_norm': 1.2936758893280629,
 'abs_sum_d_cap_edge_strength': 0.001623396566340714,
 'abs_max_d_cap_pair_norm': 0.909090909090909,
 'abs_max_d_cap_edge_strength': 0.00031204493447056374,
 'n_edges_changed_pair': 6,
 'n_edges_changed_strength': 10,
 'abs_sum_d_cap_unweighted_edge': 0.0,
 'abs_max_d_cap_unweighted_edge': 0.0,
 'n_edges_changed_unweighted': 0}

In [35]:
import plotly.express as px
import numpy as np

touched_set = set((u, v) for u, v in out["delta_touched"][["u", "v"]].itertuples(index=False))

df = out["delta_all"].copy()
df = df[df["d_cap_unweighted_edge"].notna()].copy()

rng = np.random.default_rng(42)
y_scale = (df["d_cap_unweighted_edge"].max() - df["d_cap_unweighted_edge"].min()) * 0.02
if y_scale == 0 or not np.isfinite(y_scale):
    y_scale = 1e-4
df["_y_jitter"] = rng.normal(0, y_scale, size=len(df))

# highlight touched edges if touched_set is available
touched = touched_set if "touched_set" in globals() else set()
df["touched"] = df.apply(lambda r: (r["u"], r["v"]) in touched, axis=1)

fig = px.scatter(
    df,
    x="d_cap_unweighted_edge",
    y="_y_jitter",
    color="touched",
    hover_data=["u", "v", "d_cap_unweighted_edge"],
    title="Jittered scatter of Δ cap_unweighted_edge (highlight touched edges)",
)
fig.update_traces(marker=dict(size=8, opacity=0.85))
fig.update_yaxes(visible=False)
fig.add_vline(x=0, line_dash="dash", line_color="gray")
fig.show()

In [36]:
import plotly.express as px
import numpy as np

touched_set = set((u, v) for u, v in out["delta_touched"][["u", "v"]].itertuples(index=False))

df = out["delta_all"].copy()
df = df[df["d_cap_edge_strength"].notna()].copy()

rng = np.random.default_rng(42)
y_scale = (df["d_cap_edge_strength"].max() - df["d_cap_edge_strength"].min()) * 0.02
if y_scale == 0 or not np.isfinite(y_scale):
    y_scale = 1e-4
df["_y_jitter"] = rng.normal(0, y_scale, size=len(df))

# highlight touched edges if touched_set is available
touched = touched_set if "touched_set" in globals() else set()
df["touched"] = df.apply(lambda r: (r["u"], r["v"]) in touched, axis=1)

fig = px.scatter(
    df,
    x="d_cap_edge_strength",
    y="_y_jitter",
    color="touched",
    hover_data=["u", "v", "d_cap_edge_strength"],
    title="Jittered scatter of Δ cap_edge_strength (highlight touched edges)",
)
fig.update_traces(marker=dict(size=8, opacity=0.85))
fig.update_yaxes(visible=False)
fig.add_vline(x=0, line_dash="dash", line_color="gray")
fig.show()